In [5]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.figure_factory as ff
from sklearn.preprocessing import StandardScaler
from statsmodels.tsa.seasonal import seasonal_decompose
from neuralprophet import NeuralProphet, set_log_level
from mlforecast import MLForecast

from window_ops.rolling import rolling_mean, rolling_max, rolling_min
#import seaborn as sns
import xgboost as xgb
import lightgbm as lgb
#from catboost import CatBoostRegressor
from data_factory import holiday_dates
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.fft
from layers.Embed import DataEmbedding
from layers.Conv_Blocks import Inception_Block_V1
import torch
from torch.utils.data import Dataset, DataLoader
import os
import time
import pickle
result_path = 'results/'
log_path = 'logs/'
root_path = os.getcwd()
data_path = 'data/'

In [6]:
def preprocess_wind_plant_data(df):
    
    df = df.copy()
    df.drop_duplicates(inplace=True)
    df.reset_index(inplace=True, drop=True)     

    df['datetime'] = pd.to_datetime(df['datetime'])
    date_range = pd.date_range(df['datetime'].min(), df['datetime'].max(), freq='H')
    n_missing = len(date_range) - len(df)
    
    if n_missing != 0:
        print(f">>> {n_missing} records are missing, which will be filled with interpolation")
        df.set_index('datetime', inplace=True)
        new_df = pd.DataFrame(np.nan, index = date_range, columns = df.columns)
        new_df.index.name = 'datetime'
        new_df.loc[df.index] = df.values
        df = new_df.interpolate().reset_index()
    
    #df['year'] = df['datetime'].dt.year
    df['cos_month'] = np.cos(2 * np.pi * df['datetime'].dt.month / 12)
    df['cos_day'] = np.cos(2 * np.pi * df['datetime'].dt.dayofyear / 365)
    df['cos_hour'] = np.cos(2 * np.pi * df['datetime'].dt.hour / 24)
    
    df['cos_windDir10m'] = np.cos(2 * np.pi * df['windDir10m'] / 360)
    df['cos_windDir100m'] = np.cos(2 * np.pi * df['windDir100m'] / 360)
    df.drop(['windDir10m', 'windDir100m'], axis=1, inplace=True)
    
    df.rename(columns={"datetime": "ds", target: "y"}, inplace=True)
    
    return df


In [7]:
def get_neuralprophet_model(n_lags, n_feature_lags):
    m = NeuralProphet(n_lags=n_lags, 
                      trainer_config = {"accelerator":"gpu"}
                     ) 
    for var in X_train_sub.columns:
        if var not in ['ds', 'y']:
            m.add_lagged_regressor(var, n_lags=n_feature_lags) 
            
    return m


def neuralprophet_recursive_future_prediction(model, X_train, X_test, n_lags, horizon=1):
    st = time.time()

    X_test_ = X_test.drop('y', axis=1) if 'y' in X_test.columns else X_test
    target_df = pd.concat((X_train, X_test_)).set_index('ds')

    lag_time_diff = pd.Timedelta(hours=n_lags)
    val_preds = []
    horizon = 1

    for i, d in enumerate(X_test['ds']):
        if i % horizon == 0:
         
            future = target_df.loc[d-lag_time_diff:d +pd.Timedelta(hours=horizon-1)]
            forecast = model.predict(df=future.reset_index())
            p = np.diag(forecast.loc[n_lags:, ['yhat'+str(i) for i in range(1,horizon+1)]].values)
            target_df.loc[d:d+pd.Timedelta(hours=horizon-1), 'y'] = p
            val_preds += list(p)

    et = time.time()
    pred_time = et - st
    return np.array(val_preds), target_df, pred_time


# Reading Data

In [8]:
dt_names = ['electricity_demand', 'wind_plant']
dt_name = dt_names[1]
n_lags = seq_len = 24
horizon = 1
freq = 'H'

if dt_name == 'wind_plant':
    target = 'prod' 
    preprocess_func = preprocess_wind_plant_data
else:
    target = 'demand' 
    preprocess_func = preprocess_electricty_demand_data

train_df = pd.read_csv(data_path + f'{dt_name}/train.csv', index_col=0)
future_df = pd.read_csv(data_path + f'{dt_name}/test.csv', index_col=0)

X_train = preprocess_func(train_df)
X_future = preprocess_func(future_df)
X_future.drop('y', axis=1, inplace=True)

X_val = X_train.iloc[-X_train.shape[0]//10:].copy()
X_train_sub = X_train.drop(X_val.index)

# X_test = X_val[-X_val.shape[0]//2:].copy()
# X_val.drop(X_test.index, inplace=True)

In [9]:
m, s = X_train['y'].mean(), X_train['y'].std()

In [10]:
0.15 * s + m

20.073919495123825

# Training

In [146]:
models = []
n_lags_lst = [1,3]
n_feature_lags_lst = [1, 3]


for n_lags in n_lags_lst:
    for n_feature_lags in n_feature_lags_lst:
        m = get_neuralprophet_model(n_lags, n_feature_lags)
        m_metrics = m.fit(df=X_train_sub, validation_df = X_val, freq=freq, progress="plot-all")
        models.append({'model':m, 
                       'specs': (n_lags, n_feature_lags),
                       'training_metrics': m_metrics})

WARNING - (NP.forecaster.fit) - When Global modeling with local normalization, metrics are displayed in normalized scale.
INFO - (NP.df_utils._infer_frequency) - Major frequency H corresponds to 99.996% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - H
INFO - (NP.config.init_data_params) - Setting normalization to global as only one dataframe provided for training.
INFO - (NP.config.set_auto_batch_epoch) - Auto-set batch_size to 64
INFO - (NP.config.set_auto_batch_epoch) - Auto-set epochs to 83


Finding best initial lr:   0%|          | 0/259 [00:00<?, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [208]:
n_lags_lst = [2]
n_feature_lags_lst = [1,2,3]

for n_lags in n_lags_lst:
    for n_feature_lags in n_feature_lags_lst[2:]:
        m = get_neuralprophet_model(n_lags, n_feature_lags)
        m_metrics = m.fit(df=X_train_sub, validation_df = X_val, freq=freq, progress="plot-all")
        models.append({'model':m, 
                       'specs': (n_lags, n_feature_lags),
                       'training_metrics': m_metrics})

WARNING - (NP.forecaster.fit) - When Global modeling with local normalization, metrics are displayed in normalized scale.
INFO - (NP.df_utils._infer_frequency) - Major frequency H corresponds to 99.996% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - H
INFO - (NP.config.init_data_params) - Setting normalization to global as only one dataframe provided for training.
INFO - (NP.config.set_auto_batch_epoch) - Auto-set batch_size to 64
INFO - (NP.config.set_auto_batch_epoch) - Auto-set epochs to 83


Finding best initial lr:   0%|          | 0/259 [00:00<?, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

# Validation with Recursive Forecasting

In [ ]:
for model_record in models:
    model = model_record['model']
    n_lags = model_record['specs'][0]
    val_preds, target_df, pred_time = neuralprophet_recursive_future_prediction(model, X_train_sub.copy(), X_val.copy(), n_lags)
    val_mae = np.mean(np.abs(X_val['y'] - val_preds))
    model_record['full_val_mae'] = val_mae
    model_record['val_preds'] = val_preds
    model_record['target_df'] = target_df
    model_record['pred_time'] = pred_time


# Training the Best Model (as Final Model) using All Labeled Data

In [244]:
for i, model_record in enumerate(models):
    print("model id:", i, "full_val_mae:", model_record['full_val_mae'], " | specs:", model_record['specs'])
    pred = np.array(model_record['val_preds'])
    y = X_val['y']
    df = pd.DataFrame(np.array((pred, X_val['y'])).T, columns=['pred', target], index=X_val.index)

    fig = px.line(df) 
    fig.show()


#selecting the best model specs
best_model_idx = np.argmin([model_record['full_val_mae'] for model_record in models])
best_model = models[best_model_idx]
n_lags, n_feature_lags = best_model['specs']

print('best model:', best_model_idx, best_model['specs'])

model id: 0 full_val_mae: 9.80146009117862  | specs: (1, 1)


model id: 1 full_val_mae: 10.31509250027741  | specs: (1, 3)


model id: 2 full_val_mae: 9.678437540214894  | specs: (3, 1)


model id: 3 full_val_mae: 9.636721187687023  | specs: (3, 3)


model id: 4 full_val_mae: 9.234939347588195  | specs: (2, 1)


...

model id: 7 full_val_mae: 9.75090809067498  | specs: (2, 1)


model id: 8 full_val_mae: 9.860260362925985  | specs: (2, 2)


model id: 9 full_val_mae: 9.911108586059157  | specs: (2, 3)


model id: 10 full_val_mae: 8.518658507377822  | specs: (24, 1)


best model: 10 (24, 1)


In [217]:
final_model = get_neuralprophet_model(n_lags, n_feature_lags)
final_model_metrics = final_model.fit(df=X_train, freq=freq, progress="plot-all")
models.append({'model':final_model, 
               'specs': (n_lags, n_feature_lags),
               'training_metrics': final_model_metrics})
        

WARNING - (NP.forecaster.fit) - When Global modeling with local normalization, metrics are displayed in normalized scale.
INFO - (NP.df_utils._infer_frequency) - Major frequency H corresponds to 99.996% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - H
INFO - (NP.config.init_data_params) - Setting normalization to global as only one dataframe provided for training.
INFO - (NP.config.set_auto_batch_epoch) - Auto-set batch_size to 64
INFO - (NP.config.set_auto_batch_epoch) - Auto-set epochs to 80


Finding best initial lr:   0%|          | 0/260 [00:00<?, ?it/s]

Training: 0it [00:00, ?it/s]

# Recursive Forecasting for Future Dataset (test.csv)

In [253]:
future_preds, target_df, pred_time = neuralprophet_recursive_future_prediction(final_model, X_train.copy(), X_future.copy())
models[-1]['full_val_mae'] = best_model['full_val_mae']
models[-1]['future_target_df'] = target_df
models[-1]['future_preds'] = future_preds
models[-1]['future_pred_time'] = pred_time

INFO - (NP.df_utils._infer_frequency) - Major frequency H corresponds to 96.0% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - H
INFO - (NP.df_utils._infer_frequency) - Major frequency H corresponds to 96.0% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - H


Predicting: 411it [00:00, ?it/s]

INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column
INFO - (NP.df_utils._infer_frequency) - Major frequency H corresponds to 96.0% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - H
INFO - (NP.df_utils._infer_frequency) - Major frequency H corresponds to 96.0% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - H


Predicting: 411it [00:00, ?it/s]

INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column
INFO - (NP.df_utils._infer_frequency) - Major frequency H corresponds to 96.0% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - H
INFO - (NP.df_utils._infer_frequency) - Major frequency H corresponds to 96.0% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - H


...

INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column
INFO - (NP.df_utils._infer_frequency) - Major frequency H corresponds to 96.0% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - H
INFO - (NP.df_utils._infer_frequency) - Major frequency H corresponds to 96.0% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - H


Predicting: 411it [00:00, ?it/s]

INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column
INFO - (NP.df_utils._infer_frequency) - Major frequency H corresponds to 96.0% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - H
INFO - (NP.df_utils._infer_frequency) - Major frequency H corresponds to 96.0% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - H


Predicting: 411it [00:00, ?it/s]

INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column


In [254]:
px.line(future_preds)

FigureWidgetResampler({
    'data': [{'hovertemplate': 'variable=0<br>index=%{x}<br>value=%{y}<extra></extra>',
              'legendgroup': '0',
              'line': {'color': '#636efa', 'dash': 'solid'},
              'marker': {'symbol': 'circle'},
              'mode': 'lines',
              'name': '0',
              'orientation': 'v',
              'showlegend': True,
              'type': 'scatter',
              'uid': '3853f0db-cd86-4d6f-a959-858d318e796a',
              'x': array([  0,   1,   2, ..., 741, 742, 743], dtype=int64),
              'xaxis': 'x',
              'y': array([11.06212521, 10.69043541, 10.7246933 , ..., 10.57389545, 10.52237797,
                          10.15037918]),
              'yaxis': 'y'}],
    'layout': {'legend': {'title': {'text': 'variable'}, 'tracegroupgap': 0},
               'margin': {'t': 60},
               'template': '...',
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'title': {'text': 'index'}},
               'y

# Saving Results and Models

In [259]:
keys = ['specs', 'training_metrics', 'full_val_mae', 'val_preds', 'target_df', 'pred_time']
lst_model_record_for_save = []

for i, model_record in enumerate(models):
    
    if i == len(models) - 1:
       
        model_record_for_save = {key:model_record[key]  for key in keys if key not in ['target_df', 'val_preds', 'pred_time']}
        model_record_for_save["future_target_df"] = model_record["future_target_df"]
        model_record_for_save["future_preds"] = model_record["future_preds"]
        model_record_for_save["future_pred_time"] = model_record["future_pred_time"]
        model_record_for_save['is_final'] = True
    else:
        model_record_for_save = {key:model_record[key]  for key in keys}
        model_record_for_save['is_final'] = False
            
    lst_model_record_for_save.append(model_record_for_save)
  

with open(result_path + f'neural_prophet_{dt_name}_results.pkl', 'wb') as f:
    pickle.dump(lst_model_record_for_save, f)

In [260]:
keys = ['specs', 'training_metrics', 'full_val_mae', 'val_preds', 'target_df', 'pred_time']
lst_model_record_for_save = []

for i, model_record in enumerate(models):

    
    if i == len(models) - 1:
  
        model_record_for_save = {key:model_record[key]  for key in keys if key not in ['target_df', 'val_preds', 'pred_time']}
        model_record_for_save["future_target_df"] = model_record["future_target_df"]
        model_record_for_save["future_preds"] = model_record["future_preds"]
        model_record_for_save["future_pred_time"] = model_record["future_pred_time"]
        model_record_for_save['is_final'] = True
    else:
        model_record_for_save = {key:model_record[key]  for key in keys}
        model_record_for_save['is_final'] = False
            
    model_record_for_save['model'] = model_record['model'].model
    
    lst_model_record_for_save.append(model_record_for_save)
    

with open(log_path + f'neural_prophet_{dt_name}_logs.pkl', 'wb') as f:
    pickle.dump(lst_model_record_for_save, f)